In [1]:
"""
For dimensionality project: get slices from Duke breast MRI dataset
for binary classification task, of either including tumor or not
"""
import pandas as pd
import os
import numpy as np
import pydicom
from skimage.io import imsave
from tqdm import tqdm

In [2]:
# constants
#print(os.listdir(os.getcwd()))
boxes_path = 'maps/Annotation_Boxes.csv'
data_path = None
target_png_dir = 'png_out'
if not os.path.exists(target_png_dir):
    os.makedirs(target_png_dir)

label_dirs = ['pos', 'neg']
for dr in label_dirs:
    if not os.path.exists(os.path.join(target_png_dir, dr)):
        os.makedirs(os.path.join(target_png_dir, dr))

mapping_path = 'maps/Breast-Cancer-MRI-filepath_filename-mapping.csv'
boxes_df = pd.read_csv(boxes_path)
display(boxes_df)

N_vols = len(boxes_df)
print(N_vols)

import warnings
warnings.filterwarnings('ignore')

,Patient ID,Start Row,End Row,Start Column,End Column,Start Slice,End Slice
0,Breast_MRI_001,234,271,308,341,89,112
1,Breast_MRI_002,251,294,108,136,59,72
2,Breast_MRI_003,351,412,82,139,96,108
3,Breast_MRI_004,262,280,193,204,86,95
4,Breast_MRI_005,188,213,138,178,76,122
...,...,...,...,...,...,...,...
917,Breast_MRI_918,345,395,338,395,62,85
918,Breast_MRI_919,285,312,369,397,98,109
919,Breast_MRI_920,172,193,337,355,87,101
920,Breast_MRI_921,328,374,404,446,97,121


922


In [3]:
mapping_df = pd.read_csv(mapping_path)
# in mapping csv, row order is:
# { [post1 slices], ... [post_n slices],
# pre fat-saturated slices (what we want),
# T1 slices }

# only keep rows that we need (fat-saturated "pre" exam only)
mapping_df = mapping_df[mapping_df['original_path_and_filename'].str.contains('pre')]
display(mapping_df)

,sop_instance_UID,original_path_and_filename,classic_path,descriptive_path,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,series_sort
640,1.3.6.1.4.1.14519.5.2.1.1622543824533841316221...,DICOM_Images/Breast_MRI_001/pre/Breast_MRI_001...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,1.3.6.1.4.1.14519.5.2.1.2095737470789034615864...,DICOM_Images/Breast_MRI_001/pre/Breast_MRI_001...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,1.3.6.1.4.1.14519.5.2.1.7648352393594634493272...,DICOM_Images/Breast_MRI_001/pre/Breast_MRI_001...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,1.3.6.1.4.1.14519.5.2.1.2333724865431423854264...,DICOM_Images/Breast_MRI_001/pre/Breast_MRI_001...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,1.3.6.1.4.1.14519.5.2.1.3037519639975787441522...,DICOM_Images/Breast_MRI_001/pre/Breast_MRI_001...,Duke-Breast-Cancer-MRI/Breast_MRI_001/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI001/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773061,1.3.6.1.4.1.14519.5.2.1.1620326542824048399612...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
773062,1.3.6.1.4.1.14519.5.2.1.1729373208009133770295...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
773063,1.3.6.1.4.1.14519.5.2.1.1303194464206675272654...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
773064,1.3.6.1.4.1.14519.5.2.1.3292134909528830152433...,DICOM_Images/Breast_MRI_922/pre/Breast_MRI_922...,Duke-Breast-Cancer-MRI/Breast_MRI_922/1.3.6.1....,Duke-Breast-Cancer-MRI/BreastMRI922/01-01-1990...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# def get_fixed_dcm_fnames(fname):
#     fnames = []
#     fnames.append(fname.replace('W/WO', 'WWO'))
    
#     return fnames

# def load_dcm(dcm_fname):
#     try:
#         dcm = pydicom.read_file(dcm_fname)
#     except FileNotFoundError:
#         try_fnames = get_fixed_dcm_fnames(dcm_fname)
#         for try_fname in try_fnames:
#             # try:
#             dcm = pydicom.read_file(try_fname)
#             # except FileNotFoundError:
#                         # continue
def save_dcm_slice(dcm_fname, label, vol_idx, png_path=None):
    # label = 0 if negative/healthy
    # 1 if positive
    # print(dcm_fname)
    if png_path:
        pass
    else:
        png_path = dcm_fname.split('/')[-1].replace('.dcm', '-{}.png'.format(vol_idx))
        label_dir = 'pos' if label == 1 else 'neg'
        png_path = os.path.join(target_png_dir, label_dir, png_path)
    
    if not os.path.exists(png_path):
        try:
            dcm = pydicom.dcmread(dcm_fname)
        except FileNotFoundError:
            # fix by changing *-{abc}.dcm to *-{bc}.dcm
            dcm_fname_split = dcm_fname.split('/')
            dcm_fname_end = dcm_fname_split[-1]
            assert dcm_fname_end.split('-')[1][0] == '0'
            
            dcm_fname_end_split = dcm_fname_end.split('-')
            dcm_fname_end = '-'.join([dcm_fname_end_split[0], dcm_fname_end_split[1][1:]])
            
            dcm_fname_split[-1] = dcm_fname_end
            dcm_fname = '/'.join(dcm_fname_split)
            # dcm = pydicom.read_file(dcm_fname)
            dcm = pydicom.dcmread(dcm_fname)
            
        # way 1 (simple)
        # img = dcm.pixel_array.astype('float')
        
        # way 2
        # what we did for style transfer
        obj = dcm
        if obj.PixelSpacing[0] != obj.PixelSpacing[1]:
            raise UserWarning("Different spacing {} ".format(obj.PixelSpacing))
        img = obj.pixel_array
        img_type = obj.PhotometricInterpretation

        # uint16 -> float, scaled properly for uint8
        img = img.astype(np.float) * 255. / img.max()
        # float -> uint8
        img = img.astype(np.uint8)
        if img_type == "MONOCHROME1":
            img = np.invert(img)

        # print(dcm_arr.shape)
        imsave(png_path, img)
    
    
def main():
    # iter over each patient volume
    vol_idx = -1
    # for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    for row_idx, row in tqdm(mapping_df.iterrows(), total=mapping_df.shape[0]):
        # index start at 1 here
        new_vol_idx = int((row['original_path_and_filename'].split('/')[1]).split('_')[-1])
        slice_idx = int(((row['original_path_and_filename'].split('/')[-1]).split('_')[-1]).replace('.dcm', ''))
        
        # new volume: get boxes
        if new_vol_idx != vol_idx:
            box_row = boxes_df.iloc[[new_vol_idx-1]]
            start_slice = int(box_row['Start Slice'])
            end_slice = int(box_row['End Slice'])
            assert end_slice >= start_slice 
            # print(new_vol_idx, start_slice, end_slice) 
        vol_idx = new_vol_idx
        
        # print(slice_idx)
        # if row_idx > 10000:
        #     break
        
        # fix incorrect filenames
        dcm_fname = str(row['classic_path'])
        # dcm_fname_splt = dcm_fname.split('/')
        # wrong_entry = dcm_fname_splt[1]
        # dcm_fname_splt[1] = 'Breast_MRI_{}'.format(wrong_entry[-3:])
        # dcm_fname = '/'.join(dcm_fname_splt)
        
        # print(dcm_fname)
        dcm_fname = os.path.join(data_path, dcm_fname)
        # dcm_fname = dcm_fname.replace('
        # (1) if within 3D box, save as positive
        if slice_idx >= start_slice and slice_idx < end_slice: 
            # dcm = load_dcm(dcm_fname)
            save_dcm_slice(dcm_fname, 1, vol_idx)

        # (2) if outside 3D box by >5 slices, save as negative
        elif (slice_idx + 5) <= start_slice or (slice_idx - 5) > end_slice:
            save_dcm_slice(dcm_fname, 0, vol_idx)

In [5]:
main()

100%|█████████████████████████████████| 157198/157198 [2:48:34<00:00, 15.54it/s]


In [6]:
# save_dcm_slice('/Users/nickk124/OneDrive - Duke University/research/mazurowski/dimensionality/data/dbc/manifest-1607053360376/Duke-Breast-Cancer-MRI/Breast_MRI_876/1.3.6.1.4.1.14519.5.2.1.272662101778208458825372740222864543069/1.3.6.1.4.1.14519.5.2.1.173418328215457006962502919633641763455/1-052.dcm', 
#                0, 876, '/Users/nickk124/Desktop/test6.png')